# Offsite - Featuretools

In [1]:
import pandas as pd
import numpy as np
import featuretools as ft
import matplotlib.pyplot as plt
import seaborn as sns
% matplotlib inline

In [2]:
DATA_PATH = 'Home_credit_data/'
app_train = pd.read_csv(DATA_PATH + 'application_train.csv')
# app_test = pd.read_csv(DATA_PATH + 'application_test.csv')
bureau = pd.read_csv(DATA_PATH + 'bureau.csv')
bureau_balance = pd.read_csv(DATA_PATH + 'bureau_balance.csv')
cash = pd.read_csv(DATA_PATH + 'POS_CASH_balance.csv')
credit = pd.read_csv(DATA_PATH + 'credit_card_balance.csv')
previous = pd.read_csv(DATA_PATH + 'previous_application.csv')
installments = pd.read_csv(DATA_PATH + 'installments_payments.csv')

In [3]:
# quick cleaning: unrealistic days value (365243 days is > 1000 years)
app_train = app_train.replace({365243: np.nan})
# app_test = app_test.replace({365243: np.nan})
bureau = bureau.replace({365243: np.nan})
bureau_balance = bureau_balance.replace({365243: np.nan})
cash = cash.replace({365243: np.nan})
credit = credit.replace({365243: np.nan})
previous = previous.replace({365243: np.nan})
installments = installments.replace({365243: np.nan})

### Types handling

In [4]:
# checking whether keys are of the same type (to be able to add relationships)
datasets = [app_train, bureau, bureau_balance, cash, credit, previous, installments]
keys = ['SK_ID_CURR', 'SK_ID_BUREAU', 'SK_ID_PREV']
for key in keys:
  print(key, '---')
  for dataset in datasets:
    if key in dataset:
      print(key, type(dataset[key][0]))

SK_ID_CURR ---
SK_ID_CURR <class 'numpy.float64'>
SK_ID_CURR <class 'numpy.float64'>
SK_ID_CURR <class 'numpy.float64'>
SK_ID_CURR <class 'numpy.int64'>
SK_ID_CURR <class 'numpy.float64'>
SK_ID_CURR <class 'numpy.float64'>
SK_ID_BUREAU ---
SK_ID_BUREAU <class 'numpy.float64'>
SK_ID_BUREAU <class 'numpy.int64'>
SK_ID_PREV ---
SK_ID_PREV <class 'numpy.float64'>
SK_ID_PREV <class 'numpy.int64'>
SK_ID_PREV <class 'numpy.float64'>
SK_ID_PREV <class 'numpy.float64'>


In [5]:
# harmonizing keys
for key in keys:
  for dataset in datasets:
    if key in dataset:
      dataset[key] = dataset[key].fillna(0).astype(np.int64)

In [6]:
# identifying boolean variables (minus the target variable)
app_train_types = {}
for col in app_train:
  if (app_train[col].nunique() == 2) and (app_train[col].dtype == float):
    app_train_types[col] = ft.variable_types.Boolean
del app_train_types['TARGET']
print(len(app_train_types), 'boolean variables in app_train.')

# adding ordinal variables to app_train_types
app_train_types['REGION_RATING_CLIENT'] = ft.variable_types.Ordinal
app_train_types['REGION_RATING_CLIENT_W_CITY'] = ft.variable_types.Ordinal
app_train_types['HOUR_APPR_PROCESS_START'] = ft.variable_types.Ordinal

32 boolean variables in app_train.


In [7]:
# same thing with the dataframe called previous
previous_types = {}
for col in previous:
  if (previous[col].nunique() == 2) and (previous[col].dtype == float):
    previous_types[col] = ft.variable_types.Boolean
print(len(previous_types), 'boolean variables in previous.')

2 boolean variables in previous.


In [8]:
# getting rid of the column 'SK_ID_CURR' where we don't need it
# (and we want to avoid it being treated as a variable when it's actually an ID)
installments = installments.drop(columns = ['SK_ID_CURR'])
credit = credit.drop(columns = ['SK_ID_CURR'])
cash = cash.drop(columns = ['SK_ID_CURR'])

### Entity set creation

In [9]:
# creating an entity set
es = ft.EntitySet(id='clients')

# step 1: adding the entities that have a unique key
es = es.entity_from_dataframe(entity_id='app_train',
                              dataframe=app_train,
                              index='SK_ID_CURR',
                              variable_types=app_train_types)

es = es.entity_from_dataframe(entity_id='bureau',
                              dataframe=bureau,
                              index='SK_ID_BUREAU')

es = es.entity_from_dataframe(entity_id='previous', 
                              dataframe=previous, 
                              index='SK_ID_PREV',
                              variable_types=previous_types)

# step 2: adding entities that don't have a unique key (we have to supply a name for it)
es = es.entity_from_dataframe(entity_id='bureau_balance',
                             dataframe=bureau_balance,
                             make_index=True,
                             index='bureaubalance_index')

es = es.entity_from_dataframe(entity_id='cash',
                             dataframe=cash,
                             make_index=True,
                             index='cash_index')

es = es.entity_from_dataframe(entity_id='installments',
                             dataframe=installments,
                             make_index=True,
                             index='installments_index')

es = es.entity_from_dataframe(entity_id='credit',
                             dataframe=credit,
                             make_index=True,
                             index='credit_index')

In [46]:
es

Entityset: clients
  Entities:
    app_train [Rows: 307511, Columns: 122]
    bureau [Rows: 1716428, Columns: 17]
    previous [Rows: 1670214, Columns: 37]
    bureau_balance [Rows: 27299925, Columns: 4]
    cash [Rows: 10001358, Columns: 8]
    installments [Rows: 13605401, Columns: 8]
    credit [Rows: 3840312, Columns: 23]
  Relationships:
    bureau.SK_ID_CURR -> app_train.SK_ID_CURR
    bureau_balance.SK_ID_BUREAU -> bureau.SK_ID_BUREAU
    previous.SK_ID_CURR -> app_train.SK_ID_CURR
    cash.SK_ID_PREV -> previous.SK_ID_PREV
    installments.SK_ID_PREV -> previous.SK_ID_PREV
    credit.SK_ID_PREV -> previous.SK_ID_PREV

### Adding relationships

In [11]:
# relationships step 1: defining relationships
# relationship between app_train and bureau
rel_app_bureau = ft.Relationship(es['app_train']['SK_ID_CURR'], 
                                 es['bureau']['SK_ID_CURR'])

# relationship bureau / bureau balance
rel_bureau_balance = ft.Relationship(es['bureau']['SK_ID_BUREAU'], 
                                     es['bureau_balance']['SK_ID_BUREAU'])

# relationship current app / previous apps
rel_app_previous = ft.Relationship(es['app_train']['SK_ID_CURR'], 
                                   es['previous']['SK_ID_CURR'])

# relationships between previous apps and cash, installments, and credit
rel_previous_cash = ft.Relationship(es['previous']['SK_ID_PREV'], 
                                    es['cash']['SK_ID_PREV'])

rel_previous_installments = ft.Relationship(es['previous']['SK_ID_PREV'], 
                                            es['installments']['SK_ID_PREV'])

rel_previous_credit = ft.Relationship(es['previous']['SK_ID_PREV'], 
                                      es['credit']['SK_ID_PREV'])

# relationships step 2: adding the relationships to the entity set
es = es.add_relationships([rel_app_bureau, 
                           rel_bureau_balance, 
                           rel_app_previous,
                           rel_previous_cash, 
                           rel_previous_installments, 
                           rel_previous_credit])

In [12]:
es

Entityset: clients
  Entities:
    app_train [Rows: 307511, Columns: 122]
    bureau [Rows: 1716428, Columns: 17]
    previous [Rows: 1670214, Columns: 37]
    bureau_balance [Rows: 27299925, Columns: 4]
    cash [Rows: 10001358, Columns: 8]
    installments [Rows: 13605401, Columns: 8]
    credit [Rows: 3840312, Columns: 23]
  Relationships:
    bureau.SK_ID_CURR -> app_train.SK_ID_CURR
    bureau_balance.SK_ID_BUREAU -> bureau.SK_ID_BUREAU
    previous.SK_ID_CURR -> app_train.SK_ID_CURR
    cash.SK_ID_PREV -> previous.SK_ID_PREV
    installments.SK_ID_PREV -> previous.SK_ID_PREV
    credit.SK_ID_PREV -> previous.SK_ID_PREV

### Deep feature synthesis

In [13]:
# defining primitives to use for DFS
agg_primitives =  ['sum', 'max', 'min', 'mean', 'count', 'percent_true', 'num_unique', 'mode']
trans_primitives = ['percentile', 'and']

Note: in the next cell, we could define seed features based on domain knowledge (eg the interest rate). Featuretools would then build additional features where possible.

In [14]:
# inspecting the features that will be created (only feature names and not values by passing features_only=True)
feature_names = ft.dfs(entityset=es, 
                       target_entity='app_train',
                       trans_primitives=trans_primitives,
                       agg_primitives=agg_primitives, 
                       seed_features=[],
                       max_depth=2, 
                       n_jobs=-1, 
                       verbose=1,
                       features_only=True)

Built 1820 features


In [15]:
feature_names[-10:]

[<Feature: PERCENTILE(MEAN(credit.AMT_TOTAL_RECEIVABLE))>,
 <Feature: PERCENTILE(MEAN(credit.CNT_DRAWINGS_ATM_CURRENT))>,
 <Feature: PERCENTILE(MEAN(credit.CNT_DRAWINGS_CURRENT))>,
 <Feature: PERCENTILE(MEAN(credit.CNT_DRAWINGS_OTHER_CURRENT))>,
 <Feature: PERCENTILE(MEAN(credit.CNT_DRAWINGS_POS_CURRENT))>,
 <Feature: PERCENTILE(MEAN(credit.CNT_INSTALMENT_MATURE_CUM))>,
 <Feature: PERCENTILE(MEAN(credit.SK_DPD))>,
 <Feature: PERCENTILE(MEAN(credit.SK_DPD_DEF))>,
 <Feature: PERCENTILE(COUNT(credit))>,
 <Feature: PERCENTILE(NUM_UNIQUE(credit.NAME_CONTRACT_STATUS))>]

In [58]:
# saving the features to use them in the DASK notebook
ft.save_features(feature_names, 'features.txt')

In [ ]:
# running DFS
feature_matrix, feature_names = ft.dfs(entityset=es, 
                                       target_entity='app_train',
                                       trans_primitives=trans_primitives,
                                       agg_primitives=agg_primitives, 
                                       seed_features=[],
                                       max_depth=2, 
                                       n_jobs=-1, 
                                       verbose=1,
                                       features_only=False)

feature_matrix.reset_index(inplace=True)
feature_matrix.to_csv('feature_matrix.csv', index=False)